__Toutes les fonctions définies dans le notebook pre_processing sont mises dans un fichier Python qu'on importe__

In [2]:
patients = os.listdir(INPUT_FOLDER)
patients.sort()

In [7]:
import pre_processing as pp
INPUT_FOLDER = '/home/lucsoret/Projet/Supelec/CEI/Data/sample_images'

patients = os.listdir(INPUT_FOLDER)
first_patient_path = os.path.join(INPUT_FOLDER,patients[10])
first_patient = pp.load_scan(first_patient_path)
total_images = os.listdir(first_patient_path)

image = pp.full_process(first_patient_path)

125

In [1]:
import numpy as np
import os


from keras.models import Sequential
from my_classes import DataGenerator

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

TEST_FOLDER = '/home/lucsoret/Projet/Supelec/CEI/Data/sample_images'
VAL_FOLDER = '/home/lucsoret/Projet/Supelec/CEI/Data/sample_validation_images'


# Parameters
params = {'dim_x': 10,
          'dim_y': 10,
          'dim_z': 10,
          'batch_size': 1,
          'shuffle': True}

train_folder = TEST_FOLDER
val_folder = VAL_FOLDER

#S OCCUPER DE LA PARTIE LABEL QUAND ON VA VRAIMENT L UTILISER
labels = np.array([1,0,1,0,1,1,1,0,0,1,1,0,0,0,1,1,1,1,1,0])


# Generators
training_generator = DataGenerator(**params).generate(labels, train_folder)
validation_generator = DataGenerator(**params).generate(labels, val_folder)


Using TensorFlow backend.


### Générer le modèle 

In [2]:
from keras import layers
from keras import models
from keras import Sequential

model = models.Sequential()
model.add(layers.Conv3D(32, (3, 3,3), activation='relu',
input_shape=(params['dim_z'], params['dim_x'], params['dim_y'],1)))
model.add(layers.MaxPooling3D((2, 2, 2)))
#model.add(layers.Conv3D(64, (3, 3,3), activation='relu'))
#model.add(layers.MaxPooling3D((2, 2,2)))
#model.add(layers.Conv3D(128, (3, 3, 3), activation='relu'))
#model.add(layers.MaxPooling3D((2, 2, 2)))
#model.add(layers.Conv3D(128, (3, 3, 3), activation='relu'))
#model.add(layers.MaxPooling3D((2, 2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 8, 8, 8, 32)       896       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 4, 4, 4, 32)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                32784     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 33,697
Trainable params: 33,697
Non-trainable params: 0
_________________________________________________________________


### Compile !

In [21]:
from keras import optimizers

model.compile(loss='binary_crossentropy',
optimizer=optimizers.RMSprop(lr=1e-4),
metrics=['acc'])

### Fit !

In [24]:
model.fit_generator(generator = training_generator,
                    steps_per_epoch = 20,
                    epochs = 10,
                    validation_data = validation_generator,
                    validation_steps = 20)

Epoch 1/10


StopIteration: 

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
